라이브러리 import, 학습 데이터 입력 ∙ 데이터 수정

In [7]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from bayes_opt import BayesianOptimization
from lightgbm import LGBMRegressor


In [ ]:
# 데이터 로드 
#종목코드가 포함된 열은 object형식으로 로드
stock_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_Setting.csv',low_memory=False)
stock_df_pred = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_Pred.csv',low_memory=False)

#labal 데이터프레임
label_df = stock_df['18']
#train 데이터프레임에서 label 값 삭제
stock_df = stock_df.drop({'18','7','0','1'},axis=1)
#학습에 필요한 데이터에서 Date제거
pred_tiker = stock_df_pred['0']
stock_df_pred = stock_df_pred.drop({'0','1','7'},axis=1)
# # #종목코드 int형으로 변환
# stock_df.astype({'0':'int'})
# stock_df_pred.astype({'0':'int'})

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(stock_df, label_df, test_size=0.2, random_state=42)

# SMOTE를 사용해서 데이터 불균형 교정
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)
X_test_resampled, y_test_resampled = smote.fit_resample(X_test,y_test)

LGBM 하이퍼 파라미터 튜닝(BayesianOptimization)

In [ ]:
# from bayes_opt import BayesianOptimization
# from sklearn.metrics import roc_auc_score
# from lightgbm import LGBMRegressor

# bayesian_params = {
#     'colsample_bytree': (0.0,1.0),
#     "learning_rate": (0.001, 0.3),
#     'max_depth': (1,100),
#     'min_child_samples': (1, 200), #하나의 잎에 최소 데이터 개수 (오버피팅 대응)
#     'min_child_weight': (1,20),
#     "num_leaves": (2,512),
#     'reg_alpha': (0,20),
#     'reg_lambda': (0,20),
#     'scale_pos_weight': (0.1,20),
#     'subsample': (0.1,1),
#     'feature_fraction': (0.1,1.0),
#     'bagging_fraction': (0.1,1.0),
#     'bagging_freq':(1,10)
# }

# def lgb_roc_eval(colsample_bytree,learning_rate,max_depth, min_child_samples,min_child_weight, 
#                  num_leaves, reg_lambda, reg_alpha, scale_pos_weight,subsample,feature_fraction,bagging_fraction,bagging_freq):
#   params = {
#       "n_estimators": 500,
#       'colsample_bytree': max(min(colsample_bytree, 1), 0),
#       "learning_rate": learning_rate,
#       'max_depth': int(round(max_depth)),
#       'min_child_samples': int(round(min_child_samples)),
#       'min_child_weight': int(round(min_child_weight)),
#       'num_leaves': int(round(num_leaves)),
#       'reg_lambda': max(reg_lambda, 0),
#       'reg_alpha': max(reg_alpha, 0),
#       "scale_pos_weight": scale_pos_weight,
#       'subsample': max(min(subsample, 1), 0),
#       "feature_fraction": feature_fraction,
#       'bagging_fraction': bagging_fraction,
#       'bagging_freq': round(bagging_freq)
#   }
#   lgb_model = LGBMRegressor(**params)
#   lgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric= 'mse', 
#                 early_stopping_rounds= 20)
#   valid_proba = lgb_model.predict(X_test)
#   roc_auc = roc_auc_score(y_test, valid_proba)

#   return roc_auc

# lgbBO = BayesianOptimization(f= lgb_roc_eval, pbounds=bayesian_params, random_state = 42)
# lgbBO.maximize(init_points = 5, n_iter = 25)

# # dictionary에 있는 target값을 모두 추출
# target_list = []
# for result in lgbBO.res:
#     target = result['target']
#     target_list.append(target)
# print(target_list)
# # 가장 큰 target 값을 가지는 순번(index)를 추출
# print('maximum target index:', np.argmax(np.array(target_list)))

# # 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
# max_dict = lgbBO.res[np.argmax(np.array(target_list))]
# print(max_dict)

LGBM Regressor 모델 생성 ∙ 학습

In [ ]:
# 교차검증 (StratifiedKFold)
n_split = 5
fk  = KFold(n_splits= n_split)

y_pred = np.zeros(len(X_test_resampled))

for tr_idx,val_idx in fk.split(stock_df,stock_df.index):
    X_train_resampled = stock_df.iloc[tr_idx,:]
    y_train_resampled = label_df[tr_idx]

    X_val = stock_df.iloc[val_idx,:]
    y_val = label_df[val_idx]
    
    #LGBMRegressor 모델 생성
    LGB_R = lgb.LGBMRegressor(boosting_type='gbdt',metric='mse',colsample_bytree=0.48,learning_rate=0.05,max_depth=81,min_child_samples=175,min_child_weight=19,num_leaves=268,
                            reg_alpha=10.15,reg_lambda=7.24,scale_pos_weight=0.61,subsample=0.62,bagging_fraction=0.72,bagging_freq=8,feature_fraction=0.35,num_iterations=5000,n_jobs=8,verbose=100)
                            
    LGB_R.fit(X_train_resampled,y_train_resampled,eval_set=[(X_val,y_val)],eval_metric='mse',early_stopping_rounds=50)
    y_pred += LGB_R.predict(X_test_resampled)/n_split

joblib.dump(LGB_R,'LGBM_Model.pkl')

LGBM Regressor 모델 평가 지표

In [ ]:
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n',confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

# 평가 지표 계산
preds_id = np.where(y_pred > 0.5,True,False)

get_clf_eval(y_test_resampled,preds_id)

내일 5%이상 오를 종목 예측

In [ ]:
def make_six_digit_list(input_list):
    six_digit_list = []

    for item in input_list:
        # 현재 항목의 길이를 확인
        item_str = str(item)
        item_len = len(item_str)

        if item_len < 6:
            # 6자리가 안되는 항목은 앞에 0을 채워서 6자리로 만듭니다.
            zero_padding = '0' * (6 - item_len)
            six_digit_item = zero_padding + item_str
            six_digit_list.append(six_digit_item)
        else:
            # 이미 6자리인 경우 그대로 유지
            six_digit_list.append(item_str)

    return six_digit_list

In [ ]:
# 내일 주가 예측
LGBM_M = joblib.load('LGBM_Model.pkl')
pred_X = stock_df_pred
predictions = LGBM_M.predict(pred_X,predict_disable_shape_check=True)
# 5% 이상 오를 종목 식별
threshold = 0.5
rising_stocks = [ticker for i, ticker in enumerate(pred_tiker) if predictions[i] > threshold]
rising_stocks = list(map(str, rising_stocks))
result = list(set(rising_stocks))
result = make_six_digit_list(result)
print("내일 5% 이상 상승할 종목:", result)
print(len(result))

예측한 종목 검증

In [ ]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt

def check_price_increase(stock_codes):
    result = {}
    
    for code in stock_codes:
        df = fdr.DataReader(code,'2023-10-12','2023-10-13')
        
        if len(df) < 2:
            result[code] = False
        else:
            today_close = df['Close'].iloc[-1]
            yesterday_close = df['Close'].iloc[-2]
            
            if today_close >= yesterday_close * 1.05 :
                result[code] = True
            else:
                result[code] = False
    return result

result_TF = check_price_increase(result)
result_TF = list(result_TF.values())

# True와 False의 개수를 세어 비율을 계산
true_count = sum(result_TF)
false_count = len(result_TF) - true_count

# 비율을 파이그래프로 표현
labels = ['True', 'False']
sizes = [true_count, false_count]
colors = ['green', 'red']

# 파이차트 그리기
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)

# 그래프 표시
plt.show()